# Spheres

## Background

We're given a CSV file containing rows of sphere data that we need to analyze with [`pandas`](https://pandas.pydata.org/docs/index.html). A portion of our source data looks like this:

```
pid,type,radius,more
123,m,1,more
124,b,2,more
125,m,3,more
126,b,4,more
127,m,5,more
128,m,,more
```

## Objective

We need:

- A filtered set of the original data that removes rows with missing data
- A subset of the columns
- Calculate diameter, surface area, and volume for each sphere
- Average our diameter, surface area, and volume data per sphere type
- Combine the averages with their respective counts of each type of sphere
- Write our calculated data to two files:
  - Source data for calculations
  - Final counts and averages

## Imports

It is [convention](https://pandas.pydata.org/docs/getting_started/intro_tutorials/01_table_oriented.html#what-kind-of-data-does-pandas-handle) to import `pandas` as `pd`. We'll also need the constant Pi from the `math` module.

In [1]:
import pandas as pd
from math import pi

## Helper Functions

We'll need functions to calculate the raidus, surface area, and volume of each sphere.

In [2]:
def diameter(rad):
    return rad*2

In [3]:
def surface(rad):
    return 4*pi*pow(rad,2)

In [4]:
def volume(rad):
    return (4/3)*(pi*pow(rad,3))

## Read CSV

`pandas` can read data straight from the CSV on disk with [`pandas.read_csv()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html?highlight=pandas%20read_csv#pandas.read_csv).

In [5]:
raw_spheres = pd.read_csv('spheres.csv')

Let's see what the resulting [`dataframe`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html?highlight=dataframe#pandas-dataframe) looks like:

In [6]:
print(raw_spheres)

   pid mb  rad  more
0  123  m  1.0  more
1  124  b  2.0  more
2  125  m  3.0  more
3  126  b  4.0  more
4  127  m  5.0  more
5  128  m  NaN  more


That first column is the index column and not actually in the CSV itself.

## Filtering Columns

The source CSV may have other data that you don't care about. We only care about the first three columns. [`dataframe.iloc[]`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.iloc.html) will do exactly what we want.

The ranges in `iloc[:, :3]` look strange, but from left to right we want all rows, but only the second and third columns.

In [7]:
spheres = raw_spheres.iloc[:, 1:3]
print(spheres)

  mb  rad
0  m  1.0
1  b  2.0
2  m  3.0
3  b  4.0
4  m  5.0
5  m  NaN


Or mabye you want the first three columns:

In [8]:
first_three = raw_spheres.iloc[:, :3]
print(first_three)

   pid mb  rad
0  123  m  1.0
1  124  b  2.0
2  125  m  3.0
3  126  b  4.0
4  127  m  5.0
5  128  m  NaN


## Changing Headers

Those headers could use some help... Let's rename them by modifying the dataframe's [`columns` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html?highlight=dataframe#pandas-dataframe).

In [9]:
new_headers = ["type", "radius"]
spheres.columns = new_headers
print(spheres)

  type  radius
0    m     1.0
1    b     2.0
2    m     3.0
3    b     4.0
4    m     5.0
5    m     NaN


## Missing Data

In our case, I want to drop any rows missing the `radius` data as not to have it skew our calculation later. `dataframe.dropna()` will handle that for us and update our dataframe in-place.

In [10]:
spheres.dropna(subset=["radius"], inplace=True)
print(spheres)

  type  radius
0    m     1.0
1    b     2.0
2    m     3.0
3    b     4.0
4    m     5.0


## Sorting

Say we want to sort our dataframe by type. `dataframe.sort_values()` is a lexical sort which is exactly what we want.

In [11]:
spheres.sort_values(by=["type"], inplace=True)
print(spheres)

  type  radius
1    b     2.0
3    b     4.0
0    m     1.0
2    m     3.0
4    m     5.0


> Note that each row kept its index.

## New Data

Let's add a calculated [series](https://pandas.pydata.org/docs/getting_started/intro_tutorials/01_table_oriented.html#each-column-in-a-dataframe-is-a-series) (column) for diameter, surface area, and volume. This is done by specifying the name of the series on the left assignment (`spheres["diameter"]`) and how to calculate the result on the right assignment (`diameter(spheres["radius"])`).

### Diameter

In [12]:
spheres["diameter"] = diameter(spheres["radius"])
print(spheres["diameter"])

1     4.0
3     8.0
0     2.0
2     6.0
4    10.0
Name: diameter, dtype: float64


### Surface Area

In [13]:
spheres["surface area"] = surface(spheres["radius"])
print(spheres["surface area"])

1     50.265482
3    201.061930
0     12.566371
2    113.097336
4    314.159265
Name: surface area, dtype: float64


### Volume

In [14]:
spheres["volume"] = volume(spheres["radius"])
print(spheres["volume"])

1     33.510322
3    268.082573
0      4.188790
2    113.097336
4    523.598776
Name: volume, dtype: float64


### Checkpoint

This is what the fully calculated dataframe looks like so far:

In [15]:
print(spheres)

  type  radius  diameter  surface area      volume
1    b     2.0       4.0     50.265482   33.510322
3    b     4.0       8.0    201.061930  268.082573
0    m     1.0       2.0     12.566371    4.188790
2    m     3.0       6.0    113.097336  113.097336
4    m     5.0      10.0    314.159265  523.598776


## Averages

We can now calculate the average diameter, surface area, and volume for all M and B spheres. Select a list of columns to include in the resulting dataframe, how to group that selection or results, and what calculation to apply to the result.

> Note: You **must** select what you're grouping by. In other words, if we do not _select_ `type` in the list below, then we will not be able to _groupby()_ it.

In [16]:
averages = (
    spheres[["type","diameter","surface area","volume"]]
    .groupby(["type"])
    .mean()
)
print(averages)

      diameter  surface area      volume
type                                    
b          6.0    125.663706  150.796447
m          6.0    146.607657  213.628300


## Counting

We need to know the total number of each `type` of sphere. Omitting `groupby()` will result in counting all rows in the dataframe.

In [17]:
counts = spheres.groupby(by=["type"])["diameter"].count()
print(counts)

type
b    2
m    3
Name: diameter, dtype: int64


> If you both group and count `type`, then `type` becomes an index level **and** a column label when attempting to **join** them in the next section:

```
ValueError: 'type' is both an index level and a column label, which is ambiguous.
```

## Joining Data

Say we would like to know the averages per type along with the total count of each type. [`pd.merge()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.merge.html?highlight=pandas%20merge#pandas.merge) has us covered so long as we tell it which two dataframes we want to merge and on what column to join them.

In [18]:
merged = pd.merge(counts, averages, on=["type"])
print(merged)

      diameter_x  diameter_y  surface area      volume
type                                                  
b              2         6.0    125.663706  150.796447
m              3         6.0    146.607657  213.628300


### Correct those headers

We had a `daimeter` field in both sets of data. This is likely to happen on large sets of data and sometimes you need to change these columns in place. Also, everyting after `count` is an average.

In [19]:
merged.columns = ["count", "avg diameter", "avg surface area", "avg volume"]
print(merged)

      count  avg diameter  avg surface area  avg volume
type                                                   
b         2           6.0        125.663706  150.796447
m         3           6.0        146.607657  213.628300


## File Output

Let's save our calculated data to new files with [`dataframe.to_csv()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_csv.html?highlight=to_csv#pandas.DataFrame.to_csv). The index is included by default, but we don't care about that so we'll set `index=False` to disable that behavior. The `merged` data doesn't have an index to strip.

In [20]:
spheres.to_csv('calculated_spheres.csv', index=False)
merged.to_csv('averages_spheres.csv')